In [1]:
!pip install opencv-python kociemba

In [ ]:
!pip install RubikTwoPhase

In [ ]:
!pip install pyserial

In [ ]:
import cv2
import numpy as np
#import kociemba
import twophase.solver  as sv
import serial
import time
import serial.tools.list_ports

In [ ]:
# list of serial ports available

import serial.tools.list_ports
ports = serial.tools.list_ports.comports()

for p in ports:
    print(p)

In [ ]:
"""
Methodlogy:

1) 9 circles in the middle of webcam which will capture colors of cube
2) Take those circles, make it into hsv colorspace
3) 
3) 3d array will be used as input, just convert it to string using join() method

for when we have to look at color, we can hardcode logic that identifies each new face
based on the color of center piece, and then appends it accordingly to into 3d array

For Kociemba Module

U: Yellow Center Face
L: Blue Center Face
F: Red Center Face
R: Green Center Face
B: Blue Center Face
D: White Center Face
"""

In [ ]:
class CubeScanner:

    def __init__(self):
        # accesses webcam 
        self.cap = cv2.VideoCapture(0)
        
        # setting the dimensions of webcam is necessary for color extraction logic
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

        # define circle positions
        self.center_coordinate_x = 640
        self.center_coordinate_y = 360
        self.offset_pixed_distance = 60
        self.face_order = ["Yellow", "Green", "Red", "White", "Blue", "Orange"]
        self.input_cube_array = np.empty((6,3,3), dtype=str)
        self.twophase_input_string = None




# calculated circle positions organized as a grid

    def _return_pixel_coordinates(self):
        center_coordinate_x, center_coordinate_y, offset_pixel_distance = self.center_coordinate_x, self.center_coordinate_y, self.offset_pixed_distance

        circle_coordinates = [(center_coordinate_x - offset_pixel_distance, center_coordinate_y + offset_pixel_distance), (center_coordinate_x, center_coordinate_y + offset_pixel_distance), (center_coordinate_x + offset_pixel_distance, center_coordinate_y + offset_pixel_distance), # top row
                            (center_coordinate_x - offset_pixel_distance, center_coordinate_y), (center_coordinate_x, center_coordinate_y), (center_coordinate_x + offset_pixel_distance, center_coordinate_y), # middle row
                            (center_coordinate_x - offset_pixel_distance, center_coordinate_y - offset_pixel_distance), (center_coordinate_x, center_coordinate_y - offset_pixel_distance), (center_coordinate_x + offset_pixel_distance, center_coordinate_y - offset_pixel_distance)] # bottom row

        return circle_coordinates
    
    def get_color(self, hue, sat, val):

        if sat < 50 and val > 180:
            return "W"
        elif (hue < 10 or hue > 170) and sat > 70:  # ajusted red range and added saturation check
            return "R"
        elif 10 <= hue < 25 and sat > 70 and val > 50:  # adjusted orange range and added saturation/value checks
            return "O"
        elif 25 <= hue < 35 and sat > 50:
            return "Y"
        elif 35 <= hue < 85 and sat > 40:
            return "G"
        elif 85 <= hue < 130 and sat > 40:
            return "B"
        else:
            return "U"  # unknown
        
    def scan_cube(self):

        # order of faces to scan
        face_index = 0

        # webcam logic
        while face_index < 6:

            # something new i learned
            # ret is a bool variable that returns true if the frame is available
            # frame is an image array vector captured based on default frames per second
            
            ret, frame = self.cap.read()

            # if webcame is not working
            if not ret:
                print("Failed to grab frame.")
                break

            # cv2.cv.flip(src, flipCode[, dst] )
            # 1 for dst which will mirror live stream, makes it easier to use
            frame = cv2.flip(frame, 1)

            # converting frame to hsv
            hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)


            # now add the circles onto the frame
            # we need to loop through the coordinates and draw the circles
            # for j,k in my_list (stack overflow)
            for (c_x, c_y) in self.circle_coordinates:

                cv2.circle(frame, (c_x, c_y), 7, (25, 25, 25), 2)
                pixel_center_hsv = hsv_frame[c_y, c_x]  # Changed from frame to hsv_frame
                color = self.get_color(pixel_center_hsv[0], pixel_center_hsv[1], pixel_center_hsv[2])
                
                # display the current color to the right
                cv2.putText(frame, color, (c_x + 10, c_y), cv2.FONT_HERSHEY_SIMPLEX,0.5,(0, 0, 0),2)

            # display current being displayed
            cv2.putText(frame, f"Scanning: {self.face_order[face_index]} Center Face", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            cv2.imshow("Rubik's Cube Scanner", frame)

            key = cv2.waitKey(1) & 0xFF

            # logic for taking input for cube

            # if spacebar is pressed, capture the colors
            if key == ord(' '):
                for i, (c_x, c_y) in enumerate(self.circle_coordinates):
                    print(f"Scanning face: {self.face_order[face_index]}")
                    # Images in OpenCV are represented by NumPy arrays. To access a particular image pixel, all we need to do is pass in the (x, y)-coordinates as image[y, x]:
                    pixel_center_hsv = hsv_frame[c_y,c_x]  
                    
                    color = self.get_color(pixel_center_hsv[0], pixel_center_hsv[1], pixel_center_hsv[2])

                    self.input_cube_array[face_index, i // 3, i % 3] = color
                    
                    print(f"Square {i+1}: {color}")

                face_index +=1
            
                # if all 6 sides are scanned, we are done
                if (face_index >=6):
                    print("All faces scanned.")
                    break


            # have a stop condition (click q for quit)
            if key == ord('q'):
                break

        self.cap.release()
        cv2.destroyAllWindows()
        return self.input_cube_array
    


# this class actually gets the solution for the cube with two_phase
class SolutionSolver():
    # what attributes have to be saved?

    def color_notation_to_kociemba_notation(self, str):
        notation_map = {"Y":"U", "B": "L", "R": "F", "G":"R", "O":"B", "W":"D"}
        # get method for dictionary
        return notation_map.get(str, "?")
    
    def array_to_solution(self,input_cube_array):
        self.twophase_input_string = ""

        for x in range(6):
            for y in range(3):
                for z in range(3):
                    str_temp = ""
                    str_temp = self.color_notation_to_kociemba_notation(input_cube_array[x][y][z])
                    self.twophase_input_string += str_temp

        
        try: 
            solution = sv.solve(self.twophase_input_string)
        except Exception as e:
            print(f"Error: {e}")

        return solution
    
    def two_phase_teensy(self, solved_input_string):

        split_string = solved_input_string.split() # split by whitespace
        arduino_commands = []

        for move in split_string:
            if move == '(':
                break # means we have reached the end
        
            # B3 is example of notation
            face = move[0]
            quantity = move[1]

            # according to documentation, only goes to three rotatations, remember 4 is back to original position
            if quantity == '1':
                arduino_commands.append(face)
            elif quantity == '2':
                arduino_commands.extend([face], [face])
            elif quantity == '3':
                arduino_commands.extend([face], [face], [face])

            return arduino_commands


class SerialCommunication:
    def __init__(self, port=None, baud_rate=115200):
        self.port = port
        self.baud_rate = baud_rate
        self.ser = None

    def list_ports(self):
        """List all available serial ports."""
        ports = serial.tools.list_ports.comports()
        for p in ports:
            print(p)

    def connect(self):

        if not self.port:
            self.list_ports()
            self.port = input("Enter the port the Teensy is connected to: ")
            
        try: 
            self.ser = serial.Serial(self.port, 115200, timeout=2)
            time.sleep(2) # wait for serial connection to serialize
            print(f"Successfully connected to Teensy on port: {self.port}")

        except serial.SerialException as e:
            print(f"Error opening serial port: {e}")
            # how to call a method within a class
            self.list_ports()

    def list_ports(self):
        """List all available serial ports."""
        ports = serial.tools.list_ports.comports()
        for p in ports:
            print(p)

    def send_command(self, command):
        try: 
            self.ser.write(command)
            time.sleep(0.1)
            response = self.ser.readline().decode().strip()
            print(f"Teensy response: {response}") # this is what the teensy will write to us on Python side

        except serial.SerialException as e:
            print(f"Error sending command: {e}")
    
    def close(self):
        


# put all the classes together 
class RubiksCubeRobot:
    def __init__(self):
        
        self.cube_scanner = CubeScanner()
        self.solution = SolutionSolver()
        self.serial_connection = SerialCommunication()

        def execute_solution(self, solution):


        def solve_cube(self):
            input_cube_state = self.cube_scanner.scan_cube()

            solver_output = self.solution.array_to_solution(input_cube_state)
            
            send_to_teensy = self.solution.two_phase_to_teensy(solver_output)

# main function to bring it all together








